# 4장 - 모델 훈련

지금까지는 머신러닝 모델과 훈련 알고리즘을 블랙박스처럼 취급했습니다. 앞 장의 연습문제를 풀어봤다면 내부가 어떻게 작동하는지 몰라도 많은 일을 처리할 수 있다는 점에 놀랐을 것입니다. 회귀 시스템을 최적화하고 숫자 이미지 분류기를 개선했으며 스팸 분류기를 처음부터 구축했지만 실제로 어떻게 작동하는지는 모릅니다. 많은 경우 구현의 상세 사항을 실제로 알아야할 필요는 없습니다.

하지만 어떻게 작동하는지 잘 이애하고 있으면 적절한 모델, 올바른 훈련 알고리즘, 작업에 맞는 좋은 하이퍼파라미터를 빠르게 찾을 수 있습니다. 작동 원리를 이해하고 있으면 디버깅이나 에러를 효율적으로 분석하는 데 도움이 됩니다. 이 장에서 언급하는 대부분의 주제는 신경망$^{neural \ network}$(2부에서 설명합니다)을 이해하고 구축하고 훈련시키는 데 필수입니다.

이 장에서는 가장 간단한 모델 중 하나인 선형 회귀부터 시작합니다. 이 모델을 훈련시키는 두 가지 방법을 설명하겠습니다.

- 직접 계산할 수 있는 공식을 사용하여 훈련 세트에 가장 잘 맞는 모델 파라미터(즉, 훈련 세트에 대해 비용 함수를 최고화하는 모델 파라미터)를 해석적으로 구합니다.
- 경사 하강법(GD)이라 불리는 반복적인 최적화 방식을 사용하여 모델 차라미터를 조금씩 바꾸면서 비용 함수를 훈련 세트에 대해 최소화시킵니다. 결국에는 앞의 방법과 동일한 파라미터로 수렴합니다. 경사 하강법의 변종으로 2부에서 신경망을 공부할 때 계속 사용하게 될 배치$^{batch}$ 경사 하강법, 미니배치$^{mini-batch}$ 경사 하강법, 확률적$^{stochastic}$ 경사 하강법(SGD)도 살펴보겠습니다.

그다음에는 비선형 데이터셋에 훈련시킬 수 있는 조금 더 복잡한 모델인 다항 회귀를 살펴보겠습니다. 이 모델은 선형 회귀보다 파라미터가 많아서 훈련 데이터에 과대적합되기 더 쉽습니다. 따라서 학습 곡선$^{learning \ curve}$을 사용해 모델이 과대적합되는지 감지하는 방법도 살펴보겠습니다. 그런 다음 훈련 세트의 과대적합을 감소시킬 수 있는 규제 기법을 몇 가지 알아보겠습니다.

마지막으로 분류 작업에 널리 사용하는 모델인 로지스틱 회귀와 소프트맥스 회귀를 살펴보겠습니다.

<span style="color: #FFA500">CAUTION_</span> 이 장에서는 기초적인 선형대수와 미분 기호를 사용한 수학 방정식이 꽤 나옵니다. 이 식들을 이해하려면 벡터와 행렬, 전치$^{transpose}$, 점곱, 역행렬$^{inverse \ matrix}$, 편미분$^{partial \ derivative}$에 대해 알아야 합니다. 이 개념들이 익숙하지 않다면 주피터 노트북으로 만든 선형대수와 미분에 대한 기초 튜토리얼을 깃허브( https://github.com/rickiepark/handson-ml2 )에서 살펴보세요. 정말 수학이 싫다면 이 장을 읽되 방정식은 건너뛰세요. 본문만으로도 대부분의 개념을 이해하는 데 충분히 도움 될 것입니다.

## 4.1 선형 회귀

다음은 1장에서 본 삶의 만족도에 대한 간단한 선형 회귀 모델입니다.

$$ 삶의 만족도 = \theta_0 ＋ \theta_1 × 1인당\_GDP $$

이 모델은 입력 특성인 1인당_GDP에 대한 선형 함수입니다. $\theta_0$과$\theta_1$이 모델 파라미터입니다.

더 일반적으로 선형 모델은 아래 식에서처럼 입력 특성의 가중치 합과 <span style="color: #008000">편향</span>$^{bias}$(또는 <span style="color: #008000">절편</span>$^{intercept}$)이라는 상수를 더해 예측을 만듭니다.

$$ \hat{y} = \theta_0 ＋ \theta_1x_1 ＋ \theta_2x_2 ＋ \cdots ＋ \theta_nx_n $$

- $\hat{y}$은 예측값입니다.
- $n$은 특성의 수입니다.
- $x_i$는 $i$번째 특성 값입니다.
- $\theta_j$는 $j$번째 모델 파라미터입니다(편향 $\theta_0$과 특성의 가중치 $\theta_1$, $\theta_2$, $\cdots$, $\theta_n$을 포함합니다).

이 식은 아래 식처럼 벡터 형태로 더 간단하게 쓸 수 있습니다.

$$ \hat{y} = h_0(\mathbf{x}) = \theta \cdot \mathbf{x} $$

- $\theta$는 편향 $\theta_0$과 $\theta_1$에서 $\theta_n$까지의 특성 가중치를 담은 모델의 파라미터 벡터입니다.
- $\mathbf{x}$는 $x_0$에서 $x_n$까지 담은 샘플의 <span style="color: #008000">특성 벡터</span>입니다. $x_0$은 항상 1입니다.\*
- $\theta\cdot\mathbf{x}$는 벡터 $\theta$와 $\mathbf{x}$의 점곱입니다. 이는 $\theta_0x_0＋\theta_1x_1＋\theta_2x_2＋\cdots＋\theta_nx_n$와 같습니다.
- $h_0$는 모델 파라미터 $\theta$를 사용한 가설$^{hypothesis}$ 함수입니다.

---

\* 편의상 벡터 표현으로 모델 파라미터와 특성을 모두 표현하기 위해 편향 $\theta_0$에 가상의 특성 $x_0 = 1$이 곱해졌다고 생각합니다.

<span style="color: #008000">NOTE_</span> 머신러닝에서는 종종 벡터를 하나의 열을 가진 2D 배열인 <span style="color: #008000">열 벡터</span>$^{columm \ vector}$로 나타냅니다. $\theta$와 $\mathbf{x}$가 열 벡터라면 예측은 $\hat{y}=\theta^T\mathbf{x}$입니다. 여기에서 $\theta^T$는 $\theta$의 전치$^{transpose}$입니다(열 벡터가 아니라 행 벡터가 됩니다). $\theta^T\mathbf{x}$는 $\theta^T$와 $\mathbf{x}$의 행렬 곱셈입니다. 물론 예측 결과는 같지만 스칼라 값이 아니라 하나의 원소를 가진 행렬이 만들어집니다. 이 책에서 점곱과 행렬 곱셈을 왔다 갔다 하는 것을 피하기 위해서 이 표기법을 사용하겠습니다.

이것이 선형 회귀 모델입니다. 그러면 이제 훈련시켜볼까요? 모델을 훈련시킨다는 것은 모델이 훈련 세트에 가장 잘 맞도록 모델 파라미터를 설정하는 것입니다. 이를 위해 먼저 모델의 훈련 데이터에 얼마나 잘 들어맞는지 측정해야 합니다. 2장에서 회귀에 가장 널리 사용된 성능 측정 지표는 평균 제곱근 오차(RMSE)라고 언급했습니다. 그러므로 선형 회귀 모델을 훈련시키려면 RMSE를 최소화 하는 $\theta$를 찾아야 합니다. 실제로는 RMSE보다 평균 제곱 오차$^{mean \ square \ error}$(MSE)를 최소화하는 것이 같은 결과를 내면서(어떤 함수를 최소화하는 것은 그 함수의 제곱근을 최소화하는 것과 같으므로) 더 간단합니다.\*

훈련 세트 $\mathbf{X}$에 대한 선형 회귀 가설 $h_{\theta}$의 MSE는 다음 식처럼 계산합니다.

$$ MSE(\mathbf{X}, h_{\theta}) = \frac{1}{m}\sum_{i=1}^m (\theta^T\mathbf{x}^{(i)} － y^{(i)})^2 $$

이 표기법의 대부분은 2장에서 설명했습니다. 딱 하나의 차이는 모델이 파라미터 벡터 $\theta$를 가진다는 것을 명확히 하려고 $h$ 대신 $h_{\theta}$를 사용한 것뿐입니다. 간단하게 표기하기 위해 MSE($\mathbf{X}, h_{\theta}$) 대신 MSE($\theta$)라고 쓰겠씁니다.

---

\* 최종 모델을 평가하는 데 사용하는 성능 측정 지표 말고 학습 알고리즘이 다른 함수를 최적화하는 경우가 종종 있습니다. 일반적으로 성능 측정 지표에는 없는 유용한 미분 특성이 있어서 이런 함수가 계산하기 더 쉽거나, (4.5절 '규제가 있는 선형 모델'에서 규제를 설명할 때 보겠지만) 훈련하는 동안 모델에 제약을 가하기 위해 사용합니다.

### 4.1.1 정규 방정식

비용 함수를 최소화하는 $\theta$값을 찾기 위한 <span style="color: #008000">해석적인 방법</span>이 있습니다. 다른 말로 하면 바로 결과를 얻을 수 있는 수학 공식이 있습니다. 이를 <span style="color: #008000">정규방정식</span>$^{normal \ equation}$이라고 합니다.\*

$$ \hat{\theta} = (\mathbf{X}^T\mathbf{X})^{－1}\mathbf{X}^Ty $$

- $\hat{\theta}$은 비용 함수를 최소화하는 $\theta$ 값입니다.
- $y$는 $y^{(1)}$부터 $y^{(m)}$까지 포함하는 타깃 벡터입니다.

이 공식을 테스트하기 위해 선형처럼 보이는 데이터를 생성하겠습니다

---

\* 정규방정식의 증명은 위키백과 문서( https://goo.gl/WkNEXH )를 참고하세요.